In [1]:
import pandas as pd
import numpy as np
import anndata

from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

import networkx as nx

import json
import random

In [2]:
from mapping import get_sorted_gene_order, generate_gene_id_name_map, get_master_regulator_list, get_TF_lists, get_a_matrix_threshold


from constants import OPERATIONS_DIRECTORY


In [3]:
from network_utils import get_all_targets, get_gene_targets, get_gene_targets_multi, get_influencers, calculate_net_effect, construct_network_2d, construct_network, add_network_props, insert_perturbation, activate_perturbation, update_network
from network_utils import GeneUtils
from genexpression_utils import fetch_genexpression_data, get_marker_gene_sets, fetch_fibroblast_data, fetch_myotube_data, fetch_hyb_nodes

In [4]:
thresh = 300

a_matrix_adata, b_matrix_true, c_matrix_true = get_a_matrix_threshold(thresh)

# b_matrix = anndata.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/HWG/data/HWG/operations/B_matrices_true.h5ad')

b_matrix = b_matrix_true

master_regulator_list = get_master_regulator_list()
repressorlist, activatorlist, conflictedlist, tf_list = get_TF_lists()

gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()

genes = get_sorted_gene_order()

# b_matrix_true = a_matrix_adata[a_matrix_adata.obs_names.isin(tf_list)]

gene_markers = get_marker_gene_sets()

 Total Activators : 174
Total Reprossors : 705
Total Conflicted : 729
Total Transcription Factors : 1608
total master regulators 439
 Total Activators : 174
Total Reprossors : 705
Total Conflicted : 729
Total Transcription Factors : 1608
Extracted 43945 mappings from 73467 ensembl lines
Extracted 38606 mappings from gene id gtf file
Total Mappings Extracted 59979
ordering 18582 genes


In [6]:
a_matrix_adata

AnnData object with n_obs × n_vars = 19678 × 19678
    layers: 'HURI', 'STRING', 'HWG_0', 'HWG_300'

In [13]:
import scipy.io as sio
import scipy.sparse as sp

adata = a_matrix_adata

# Extract sparse matrix (X), obs_names (cells), var_names (genes/features)
X = adata.X.tocsr() if sp.issparse(adata.X) else sp.csr_matrix(adata.X)
obs_names = adata.obs_names.to_numpy()
var_names = adata.var_names.to_numpy()

# Save as MATLAB .mat file
sio.savemat("data/a_matrix_sparse.mat", {
    "X": X,  # sparse matrix preserved
    "obs_names": obs_names,
    "var_names": var_names,
    "gene_names": [gene_id_name_map.get(i, "") for i in obs_names]
})